<a href="https://colab.research.google.com/github/christophergaughan/Bioinformatics-Code/blob/main/Copy_of_Nat_2024_Regression_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch

# Load the data into a pandas DataFrame
file_path = "/content/drive/MyDrive/Natesh_scrna/liver_TF_normalized_week7_17 (1).csv"
data = pd.read_csv(file_path)

In [ ]:
data.head()

In [ ]:
# Assuming the first column has the DNA sequence identifiers
data['Unnamed'] = data.iloc[:, 0].str.split("_").str[0]

In [ ]:
data.head()

**Problem: the strings associated with the transcription factors are not straightforward like I thought. I am missing some key concept or formulation for labelling the transcription factors, will address later.**

In [ ]:
print(data.columns.tolist())

# or a helper function:
def check_gene_presence(gene_name, df):
    variants = [gene_name.upper(), gene_name.capitalize(), gene_name.lower()]
    found = []
    for variant in variants:
        if variant in df.columns:
            found.append(variant)
    return found if found else None

print("Alb columns found:", check_gene_presence("Alb", data))
print("Afp columns found:", check_gene_presence("Afp", data))
print("Hnf4a columns found:", check_gene_presence("Hnf4a", data))


## Checking Gene Presence

Below is the code snippet used to verify whether our dataset (`data`) contains columns for **Alb**, **Afp**, and **Hnf4a** (in various case variants). We also print out the entire list of columns in the DataFrame for reference.

```python
# Print all columns in the dataset
print(data.columns.tolist())

# Define a helper function to check for possible variations of a gene name
def check_gene_presence(gene_name, df):
    variants = [gene_name.upper(), gene_name.capitalize(), gene_name.lower()]
    found = []
    for variant in variants:
        if variant in df.columns:
            found.append(variant)
    return found if found else None

# Check for variations of 'Alb', 'Afp', and 'Hnf4a'
print("Alb columns found:", check_gene_presence("Alb", data))
print("Afp columns found:", check_gene_presence("Afp", data))
print("Hnf4a columns found:", check_gene_presence("Hnf4a", data))


[  'Unnamed: 0', 'ALB', 'AFP', 'HNF4A', 'FOXA3', 'APOA1', 'HHEX', 'PROX1',   'TBX3', 'ONECUT1', 'GATA4', 'HNF1A', 'CEBPA', 'TTR', 'ATF5', 'CEBPB',   'CREB3L3', 'HLF', 'KLF15', 'MLXIPL', 'NR0B2', 'NR1H4', 'NR1I3', 'NR5A2',   'PPARA', 'RORC', 'NFIB', 'NFIC', 'PPARG', 'RARA', 'RORA', 'PARP10',   'PARP12', 'MAFB', 'CRY1', 'CUX2', 'ARNTL', 'KAT2B', 'STAT5A', 'ABTB2',   'ATOH8', 'CRIP2', 'MAOA', 'HLX', 'SIRT5', 'PAX8', 'ARID5A', 'L3MBTL4',   'TBX15', 'AR', 'NFIX', 'NFIA', 'RXRA', 'FOXA1', 'ESR1', 'FOXA2',   'HNF1B', 'cell_state', 'Unnamed']

Alb columns found: ['ALB']

Afp columns found: ['AFP']

Hnf4a columns found: ['HNF4A']


In [ ]:
data.columns

### so I can have the notebook organized like I did last time I will move the 'cell_state' column to the zeroth column

In [ ]:
# Remove the 'cell_state' column from its current position
cell_state_series = data.pop('cell_state')

# Insert 'cell_state' back as the first column
data.insert(0, 'cell_state', cell_state_series)

# Verify
data.head()


In [ ]:
# If you're certain 'Unnamed: 0' is the column name:
data.drop('Unnamed: 0', axis=1, inplace=True)

# Verify removal:
print(data.columns)


In [ ]:
data.head()

In [ ]:
# Assuming the first column has the DNA sequence identifiers
data['cell_state'] = data.iloc[:, 0].str.split("_").str[0]

In [ ]:
data.head()

**Explanation**
`(cell)(\d+)`:

Captures two groups:

* Group 1: The word "cell"
* Group 2: One or more digits `(\d+)`

    * `r'\1_\2'`: Replaces with Group 1, then an underscore
    * then Group 2 `regex=True`: Ensures we’re using a regular-expression replacement (necessary in recent versions of pandas)

In [ ]:
import re

data['cell_state'] = data['cell_state'].str.replace(
    r'(cell)(\d+)',
    r'\1_\2',
    regex=True
)


In [ ]:
data.head()

### Want to see if their is an approximate linear pattern in the numbers

example workflow to extract just the integer portion from "cell_XXX" (e.g., "cell_183" → 183), store it in a new column, and plot it in the order of the rows.

**Explanation**

* Regex Extraction:
    * `str.extract(r'cell_(\d+)')` captures only the digits after "cell_".
* The regular expression `r'cell_(\d+)'` means:
`"cell_":` literally the string `“cell_”`
* `(\d+)`: capture one or more digits into a group

**Converting to Integer:**
* `.astype(int)` ensures we treat the extracted string digits as numerical values, making it easier to plot or do numerical operations.

**Plotting:**
* `plt.plot(data['cell_number'])` creates a line plot of the values in the order they appear.
* We could also use a scatter plot if you prefer: `plt.scatter(datas.index, datas['cell_number'])`.

This way, we can quickly see if the cell numbers follow any pattern across rows (e.g., if they increase steadily, jump around, etc.).










In [ ]:
import re
import matplotlib.pyplot as plt

# 1. Extract the numeric part (digits) from the 'cell_state' column
#    For example, "cell_183" → 183
data['cell_number'] = data['cell_state'].str.extract(r'cell_(\d+)', expand=False)

# 2. Convert that numeric part to an integer (if desired)
data['cell_number'] = data['cell_number'].astype(int)

# 3. Plot these numbers in the order they appear (row-wise)
plt.figure(figsize=(10, 5))
plt.plot(data['cell_number'], marker='o')
plt.title("Cell Number in Row Order")
plt.xlabel("Row Index")
plt.ylabel("Cell Number")
plt.show()


## Which TFs Should We Include?
### A. Check for the “classic” TFs I used last year
Last year, I tried:

`['Foxa2', 'Foxa3', 'Hnf1a', 'Cebpa', 'Tbx3', 'Prox1', 'Hnf4a']
`

In the new dataset, you see columns like:

`'FOXA1', 'FOXA2', 'FOXA3', 'HNF1A', 'HNF1B', 'HNF4A', 'ONECUT1' (≅ HNF6), 'GATA4', 'HHEX' (≅ HEX), 'CEBPA', 'PROX1', 'TBX3', ...`

So all of those seven from last year are indeed present in some form (watching out for uppercase). Specifically:

Foxa2 → `'FOXA2'`
Foxa3 → `'FOXA3'`
Hnf1a → `'HNF1A'`
Cebpa → `'CEBPA'`
Tbx3 → `'TBX3'`
Prox1 → `'PROX1'`
Hnf4a → `'HNF4A'`

### B. Consider adding other related TFs in this dataset
* FOXA1: sometimes interesting when analyzing hepatocyte or endoderm lineage development.
* HNF1B: the new data has `'HNF1B'`1.
* GATA4: in the list of columns. Often relevant for liver development.
* HHEX: in the data as `'HHEX'` (a known alias for “Hex”).
* ONECUT1 (alias for HNF6) if we want to replicate the idea of HNF6.
* CEBPB if we’re curious about other members of the CEBP family.

If Natesh/Dan specifically want to see all TFs that have positive correlation with ALB or negative correlation with AFP, we can do a broader correlation test across all columns that look like TFs (there are many: ARNTL, ESR1, RORA, etc.). But usually, one focuses on a curated list—especially if these TFs are known from previous hepatic-development studies.





## Workflow to Focus on “Positive Correlation with ALB or Negative Correlation with AFP

### Non-Neural-Netowork: Pearsons Correlation”

Create a subset DataFrame with your TFs + ALB + AFP + HNF4A



In [ ]:
import pandas as pd

# these are the TFs we will work with in this dataset:last year plus FOXA1 (including FOXA1 since it has a role in hepatocyte development)
tf_list = [
    'FOXA1', 'FOXA2', 'FOXA3', 'HNF1A', 'CEBPA',
    'PROX1', 'TBX3', 'HNF4A'
]

# We'll also include ALB, AFP, and HNF4A for correlation
cols_of_interest = tf_list + ['ALB', 'AFP', 'HNF4A']  # 'HNF4A' is both TF & "target"

df_sub = data[cols_of_interest].copy()


**make sure to get rid of div/0 errors**

In [ ]:
import numpy as np
import pandas as pd

# List of columns to drop:
# - data.columns[0] (the 0th column, e.g. 'cell_state')
# - 'Unnamed' (the extra column name, if it exists)
# - 'cell_number' (the numeric part you extracted but no longer need in the correlation)
cols_to_drop = [data.columns[0], 'Unnamed', 'cell_number']

# 1. Drop those columns (ignore errors if any don't exist)
data_cln = data.drop(columns=cols_to_drop, errors='ignore')

# 2. Replace '#DIV/0!' with NaN
data_cln = data_cln.replace('#DIV/0!', np.nan)

# 3. Convert columns to numeric (coerce invalid values to NaN)
for col in data_cln.columns:
    data_cln[col] = pd.to_numeric(data_cln[col], errors='coerce')

# 4. Now compute correlation among all numeric columns
corr_matrix = data_cln.corr(method='pearson')

print(corr_matrix)


## Compute the Correlation Matrix

In [ ]:
import numpy as np
import pandas as pd

# Replace the '#DIV/0!' strings with NaN in df_sub
df_sub = df_sub.replace('#DIV/0!', np.nan)

# Convert every column to numeric, coercing invalid to NaN
for col in df_sub.columns:
    df_sub[col] = pd.to_numeric(df_sub[col], errors='coerce')


In [ ]:
print(df_sub.dtypes)


This TypeError often arises when pandas detects duplicate column names, causing `df_sub[col]` to return a DataFrame instead of a Series. In the error output, notice that 'HNF4A' is listed twice in the dtypes:

FOXA1    float64

FOXA2    float64

FOXA3    float64

HNF1A    float64

CEBPA    float64

PROX1    float64

TBX3     float64

HNF4A     object   <─ 1st HNF4A

ALB       object

AFP       object

HNF4A     object   <─ 2nd HNF4A

dtype: object



## Rename One of the Duplicated Columns
maybe we want to keep both columns for some reason (e.g., they are slightly different “HNF4A” metrics), rename one

In [ ]:
# Suppose we rename the second "HNF4A" to "HNF4A_2"
dup_mask = df_sub.columns.duplicated(keep='first')
# This mask is True for any duplicate columns after the first occurrence

for i, dup in enumerate(dup_mask):
    if dup:
        old_col = df_sub.columns[i]
        df_sub.rename(columns={old_col: old_col + "_2"}, inplace=True)

# Now no columns share the exact same name
# so "HNF4A" might become "HNF4A_2" for the second copy.


In [ ]:
for col in df_sub.columns:
    df_sub[col] = pd.to_numeric(df_sub[col], errors='coerce')



In [ ]:
print("df_sub columns:\n", df_sub.columns)
print("\nDuplicated columns?\n", df_sub.columns.duplicated(keep=False))


In [ ]:
df_sub["HNF4A_2"]


In [ ]:
df_sub = df_sub.loc[:, ~df_sub.columns.duplicated(keep='first')]


In [ ]:
df_sub.replace('#DIV/0!', np.nan, inplace=True)

for col in df_sub.columns:
    df_sub[col] = pd.to_numeric(df_sub[col], errors='coerce')


In [ ]:
import numpy as np
import pandas as pd

# 1. Drop duplicate columns (if any remain)
df_sub = df_sub.loc[:, ~df_sub.columns.duplicated(keep='first')]

# 2. Replace '#DIV/0!' with NaN
df_sub.replace('#DIV/0!', np.nan, inplace=True)

# 3. Convert columns to numeric where possible
for col in df_sub.columns:
    df_sub[col] = pd.to_numeric(df_sub[col], errors='coerce')

# 4. Compute Pearson correlation only on numeric columns
corr_matrix = df_sub.corr(method='pearson')


*Focus on ALB, AFP, and HNF4A*
Since PI and grad student want:

* Positive correlations with ALB
* Negative correlations with AFP
* Correlation with HNF4A

**We can pull those out**

In [ ]:
corr_with_alb = corr_matrix['ALB'].drop('ALB', errors='ignore').sort_values(ascending=False)
corr_with_afp = corr_matrix['AFP'].drop('AFP', errors='ignore').sort_values(ascending=False)
corr_with_hnf4a = corr_matrix['HNF4A'].drop('HNF4A', errors='ignore').sort_values(ascending=False)


In [ ]:
print("df_sub columns:", df_sub.columns.tolist())


In [ ]:
corr_matrix = df_sub.corr(method='pearson')
print("corr_matrix columns:", corr_matrix.columns.tolist())


In [ ]:
# Rename 'HNF4A_2' back to 'HNF4A'
df_sub.rename(columns={"HNF4A_2": "HNF4A"}, inplace=True)

# Now df_sub.columns (and your correlation matrix) will show 'HNF4A' instead.
print(df_sub.columns)


In [ ]:
corr_matrix = df_sub.corr(method='pearson')
print(corr_matrix.columns)


In [ ]:
# Example: Extract correlation with HNF4A
corr_with_hnf4a = corr_matrix["HNF4A"].drop("HNF4A", errors="ignore")
print(corr_with_hnf4a.sort_values(ascending=False))


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df_sub already has numeric columns for all TFs + ALB + AFP
corr_matrix = df_sub.corr(method='pearson')


In [ ]:
# Correlation with ALB (excluding ALB itself if present)
corr_with_alb = corr_matrix["ALB"].drop("ALB", errors="ignore")

# Correlation with AFP (excluding AFP itself if present)
corr_with_afp = corr_matrix["AFP"].drop("AFP", errors="ignore")


In [ ]:
# Sort descending so that highest correlations appear first
corr_with_alb_sorted = corr_with_alb.sort_values(ascending=False)

plt.figure(figsize=(7, 4))
sns.barplot(
    x=corr_with_alb_sorted.index,
    y=corr_with_alb_sorted.values,
    palette="Blues_r"
)
plt.title("Non-Neural-Network Correlation with ALB")
plt.ylabel("Pearson Correlation")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show();


In [ ]:
corr_with_afp_sorted = corr_with_afp.sort_values(ascending=False)

plt.figure(figsize=(7, 4))
sns.barplot(
    x=corr_with_afp_sorted.index,
    y=corr_with_afp_sorted.values,
    palette="Reds_r"
)
plt.title("Non-Neural-Network Correlation with AFP")
plt.ylabel("Pearson Correlation")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
corr_with_hnf4a = corr_matrix["HNF4A"].drop("HNF4A", errors="ignore").sort_values(ascending=False)

plt.figure(figsize=(7, 4))
sns.barplot(
    x=corr_with_hnf4a.index,
    y=corr_with_hnf4a.values,
    palette="magma"
)
plt.title("Non-Neural-Network Correlation with HNF4A")
plt.ylabel("Pearson Correlation")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


# Neural Network

# PyTorch Neural Network Analysis for TF-Based Gene Expression Prediction

In this section, we will introduce a **PyTorch**-based neural network to predict the expression of four key maturity genes (\`ALB\`, \`AFP\`, \`APOA1\`, and \`TTR\`) using the expression levels of 12 transcription factors (TFs) as inputs. These TFs include:

1. **FOXA1**  
2. **FOXA2**  
3. **FOXA3**  
4. **HNF1A**  
5. **HNF1B**  
6. **HNF4**  
7. **HNF6**  
8. **GATA4**  
9. **CEBPA**  
10. **TBX3**  
11. **PROX1**  
12. **HEX**

## Objective
To capture potentially **non-linear** and **multi-dimensional** relationships that go beyond traditional methods (e.g., Pearson correlation or linear regression). The network will output predictions for **ALB**, **AFP**, **APOA1**, and **TTR** simultaneously, allowing us to:

- Estimate how well these TFs collectively predict each gene.  
- Handle complex interactions among TFs.  
- Provide a more robust framework for modeling gene regulation patterns in scRNA-seq data.

## Approach Outline

1. **Data Preparation**  
   - **Subset** the relevant TF columns and the four maturity genes from the main DataFrame.  
   - **Clean** any invalid strings (e.g., \`#DIV/0!\`) by replacing them with missing values (\`NaN\`) and coercing to numeric.  
   - **Drop** rows containing \`NaN\` (or handle via imputation) to ensure the input matrix is valid for the model.

2. **Train/Test Split**  
   - We will split our dataset into **training** ~80% and **test** ~20% sets.  
   - This split ensures an unbiased estimate of how the model generalizes to unseen data.

3. **Neural Network Architecture**  
   - **Input Layer**: 12 neurons, one for each TF.  
   - **Hidden Layers**: Two fully connected layers (e.g., 64 units → 32 units), each followed by a ReLU activation.  
   - **Output Layer**: 4 neurons, corresponding to \`ALB\`, \`AFP\`, \`APOA1\`, and \`TTR\`.  
   - **Loss Function**: Mean Squared Error (MSE) to measure how close the predictions are to the actual gene expression.  
   - **Optimizer**: Adam (with a default learning rate of 1e-3) for efficient gradient-based optimization.

4. **Training**  
   - **Epochs**: Typically 30-50 to start, adjusting based on validation loss curves.  
   - **Batch Size**: 32 (tunable if we see memory or convergence issues).  
   - **Loss Monitoring**: We will track the training loss per epoch, ensuring the model is learning without overfitting.

5. **Evaluation**  
   - **Test Set MSE**: To quantify predictive performance for each gene.  
   - **R² and Correlation**: To measure how well the model explains variance in the target genes (and compare with simpler methods).  
   - **Interpretability**: Consider advanced methods like permutation importance or SHAP analysis if we need deeper insight into TF contributions.

By training this **multi-output** neural network, we aim to capture richer patterns in the transcriptomic data and potentially unveil complex gene regulatory interactions that a simpler correlation-based approach might overlook.


### **Last year** Define Columns of Interest as we did last year
According to last years objective, you have 12 TFs and 4 genes representing differentiation:



In [ ]:
print(data.columns.tolist())


From the columns list, we see:

* `HNF4A` is present (instead of `HNF4`).
* `ONECUT1` is present (commonly the alias for `HNF6`).
* `HHEX` is present (commonly the alias for `HEX`).

So, `HNF4`, `HNF6`, and `HEX` are not in our DataFrame, but `HNF4A`, `ONECUT1`, and `HHEX` are. Similarly, `FOXA1`, `FOXA2`, `FOXA3`, `HNF1A`, `CEBPA`, `TBX3`, and `PROX1` do appear as you might expect.

In [ ]:
import pandas as pd
import numpy as np

# Corrected Transcription Factors to match the columns you actually have
tf_list = [
    "FOXA1",  # present
    "FOXA2",  # present
    "FOXA3",  # present
    "HNF1A",  # present
    "HNF1B",  # present
    "HNF4A",  # instead of HNF4
    "ONECUT1",# instead of HNF6
    "GATA4",  # present
    "CEBPA",  # present
    "TBX3",   # present
    "PROX1",  # present
    "HHEX"    # instead of HEX
]

# Differentiation Genes (confirmed present)
target_genes = ["ALB", "AFP", "APOA1", "TTR"]

# Combine them
cols_of_interest = tf_list + target_genes

# Now create the subset DataFrame without the KeyError
df_sub = data[cols_of_interest].copy()


## Creating a New DataFrame `data_nn` Without Unwanted Columns

To facilitate our **neural network** analysis, we'll create a new DataFrame called **`data_nn`** that excludes certain non-essential columns—namely **`'Unnamed'`, `'cell_state'`, and `'cell_number'`**. This keeps our data cleaner and ready for machine-learning tasks.

```python


In [ ]:
# Create a new DataFrame by dropping columns we don't need
cols_to_drop = ['Unnamed', 'cell_state', 'cell_number']
data_nn = data.drop(columns=cols_to_drop, errors='ignore')

# Verify the columns in data_nn
print("Columns in data_nn:")
print(data_nn.columns.tolist())


In [ ]:
# Example quick check:
print(data_nn.isna().sum())
print(data_nn[data_nn == '#DIV/0!'].count())  # if not already replaced


drop rows with error values


In [ ]:
import numpy as np
import pandas as pd

# Replace '#DIV/0!' with NaN in-place
data_nn.replace('#DIV/0!', np.nan, inplace=True)

# Convert columns to numeric where possible
for col in data_nn.columns:
    data_nn[col] = pd.to_numeric(data_nn[col], errors='coerce')

# Optional: Check how many NaNs remain
print(data_nn.isna().sum())

# If it’s only that one row, you can:
# (A) Drop that single row
data_nn.dropna(subset=data_nn.columns, inplace=True)

# OR (B) Impute with median/mean, e.g.:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy='median')
# data_imputed = imputer.fit_transform(data_nn)
# data_nn = pd.DataFrame(data_imputed, columns=data_nn.columns)


In [ ]:
print(data_nn.isna().sum())               # Missing values
print(data_nn[data_nn == '#DIV/0!'].sum())# Should be 0 occurrences


**see how many rows were dropped**

In [ ]:
# Let's say you start with some shape for data_nn
original_shape = data_nn.shape

# Identify rows that have '#DIV/0!' in any column
invalid_rows = data_nn.index[data_nn.eq('#DIV/0!').any(axis=1)]

# Number of invalid rows
num_invalid_rows = len(invalid_rows)
print(f"Number of rows containing '#DIV/0!': {num_invalid_rows}")

# Option 1: Drop those rows
data_nn_dropped = data_nn.drop(index=invalid_rows)
dropped_shape = data_nn_dropped.shape
rows_dropped = original_shape[0] - dropped_shape[0]
print(f"Rows dropped: {rows_dropped}")

# Option 2: If you replaced '#DIV/0!' with NaN first and then used dropna():
data_nn_replaced = data_nn.replace('#DIV/0!', np.nan)
data_nn_droppedna = data_nn_replaced.dropna()
rows_dropped_na = data_nn_replaced.shape[0] - data_nn_droppedna.shape[0]
print(f"Rows dropped after replacing with NaN and then dropna(): {rows_dropped_na}")


In [ ]:
print(data_nn.head(10))


# PyTorch Equivalent of the Previous TensorFlow Classification Workflow

## Multi-Output Regression Example in PyTorch

Why Multi-Output?
In short, you want to predict or model multiple gene expressions simultaneously rather than running multiple single-gene regressions. That is precisely multi-output regression. It’s beneficial because it:

* Treats the 4 target genes as a joint learning problem.
* Potentially captures shared signals and dependencies in the data.
* Reduces the overhead of training 4 separate networks.
* Aligns nicely with the “multi-task” perspective in scRNA-seq data, where multiple genes are co-expressed and co-regulated by the same set of TFs.




In [ ]:
import pandas as pd
import numpy as np

# Just to confirm what df_sub has:
print(df_sub.columns)

# Suppose we want to predict [ALB, AFP, APOA1, TTR] from the 12 TFs
tf_list = [
    "FOXA1", "FOXA2", "FOXA3",
    "HNF1A", "HNF1B", "HNF4A",
    "ONECUT1", "GATA4", "CEBPA",
    "TBX3", "PROX1", "HHEX"
]

target_genes = ["ALB", "AFP", "APOA1", "TTR"]

# X: TF columns
X = df_sub[tf_list].values  # shape: (num_samples, 12)

# y: 4 gene columns
y = df_sub[target_genes].values  # shape: (num_samples, 4)

print("X shape:", X.shape)
print("y shape:", y.shape)


Because we explicitly chose the four differentiation genes
‘
𝐴
𝐿
𝐵
‘
,
‘
𝐴
𝐹
𝑃
‘
,
‘
𝐴
𝑃
𝑂
𝐴
1
‘
,
‘
𝑇
𝑇
𝑅
‘
‘ALB‘,‘AFP‘,‘APOA1‘,‘TTR‘ as our target_genes, our y DataFrame (or array) ends up having exactly 4 columns—one for each of those genes. In other words, we're doing multi-output regression where the four columns in y represent the expression levels (or values) of those four specific target genes

In [ ]:
data_nn.replace('#DIV/0!', np.nan, inplace=True)


In [ ]:
for col in data_nn.columns:
    data_nn[col] = pd.to_numeric(data_nn[col], errors='coerce')


In [ ]:
data_nn.dropna(inplace=True)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train set:", X_train.shape, y_train.shape)
print("Test set:", X_test.shape, y_test.shape)


In [ ]:
print("X_train dtype:", X_train.dtype)
print("X_train sample:\n", X_train[:5])  # see the first 5 rows



In [ ]:
import numpy as np

# Replace '#DIV/0!' with NaN in your DataFrame
df_sub = df_sub.replace('#DIV/0!', np.nan)

# Option A: Drop rows with NaN
# df_sub.dropna(inplace=True)

# Option B: Or you can impute, e.g. fill with 0 or median
df_sub.fillna(value=np.median, inplace=True)
#  -- or use sklearn's SimpleImputer, etc.

# Now your entire df_sub should be numeric or NaN-free


In [ ]:
print(df_sub[df_sub == '#DIV/0!'].sum())  # Should all be 0
print(df_sub.isna().sum())               # Confirm minimal NaNs or 0 if you dropped/imputed


In [ ]:
print(df_sub.dtypes)  # Check which columns are "object"
print(df_sub.head())  # Inspect the first rows for those columns


In [ ]:
# Replace non-numeric strings (like '#DIV/0!') with NaN if you haven't already
df_sub.replace("#DIV/0!", np.nan, inplace=True)

# Convert each column to numeric
for col in df_sub.columns:
    df_sub[col] = pd.to_numeric(df_sub[col], errors="coerce")


In [ ]:
print(df_sub.dtypes)


In [ ]:
from sklearn.model_selection import train_test_split

X = df_sub[tf_list].values  # shape: (n_samples, n_TFs)
y = df_sub[target_genes].values  # shape: (n_samples, n_genes)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)


In [ ]:
import torch

X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_test_t  = torch.tensor(X_test,  dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
y_test_t  = torch.tensor(y_test,  dtype=torch.float32)


## What Is Multi-Output Regression?

In a typical (single-output) regression problem, you might predict **one continuous variable** $y$ from your features $x_1, x_2, \dots, x_n$. For example, in a simpler scenario, you might model **ALB** expression alone using 12 TFs.

However, **multi-output regression** (sometimes called **multi-variate regression**, in the sense of multiple dependent variables) expands this approach to:

$$
\mathbf{y} \;=\; \begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_k
\end{bmatrix},
$$

where $k$ is the number of target variables you want to predict. In your case, $k=4$ for $\{\text{ALB}, \text{AFP}, \text{APOA1}, \text{TTR}\}$.


In [ ]:
# 1. Imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# 2. Define the Columns of Interest
# Example:
tf_list = [
    "FOXA1", "FOXA2", "FOXA3",
    "HNF1A", "HNF1B", "HNF4A",
    "ONECUT1", "GATA4", "CEBPA",
    "TBX3", "PROX1", "HHEX"
]
target_genes = ["ALB", "AFP", "APOA1", "TTR"]  # Our multi-output targets

# df_sub is expected to have columns in tf_list + target_genes
print("Columns in df_sub:", df_sub.columns.tolist())


In [ ]:
# 3. Split into Features (X) and Targets (y)
X = df_sub[tf_list].values      # shape: (num_samples, 12)
y = df_sub[target_genes].values # shape: (num_samples, 4)


In [ ]:
# 4. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes: ", X_test.shape,  y_test.shape)


In [ ]:
# (Optional) Use a Dataset and DataLoader for mini-batch training
class RNARegressDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_dataset = RNARegressDataset(X_train_t, y_train_t)
test_dataset  = RNARegressDataset(X_test_t,  y_test_t)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)


In [ ]:
# 6. Define a Multi-Output Regression Network
class TFRegressor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TFRegressor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)  # 4 outputs for ALB, AFP, APOA1, TTR

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # shape: (batch_size, 4)
        return x

input_dim  = len(tf_list)      # e.g., 12 TFs
output_dim = len(target_genes) # 4 genes
model = TFRegressor(input_dim, output_dim)


In [ ]:
# 7. Set Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Bug here

In [ ]:
# 8. Training Loop
def train_model(model, train_loader, criterion, optimizer, epochs=20):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}")

train_model(model, train_loader, criterion, optimizer, epochs=20)

In [ ]:
print(df_sub.isna().sum())     # Are there any NaNs in df_sub?


In [ ]:
np.isinf(df_sub.values).any()


In [ ]:
print("Any NaNs in X_train_t?", torch.isnan(X_train_t).any())
print("Any Infs in X_train_t?", torch.isinf(X_train_t).any())
print("Any NaNs in y_train_t?", torch.isnan(y_train_t).any())
print("Any Infs in y_train_t?", torch.isinf(y_train_t).any())


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Then do train/test split on X_scaled


In [ ]:
# This shows you exactly which row(s) in df_sub have NaNs
na_rows = df_sub[df_sub.isna().any(axis=1)]
print("Rows with NaN:\n", na_rows)


In [ ]:
# Drop any rows that have NaN in any column
df_sub.dropna(axis=0, how='any', inplace=True)

# Double-check that no rows have NaN now:
print(df_sub.isna().sum())


Split into Features (X) and Targets (y)



In [ ]:
# Features: TFs
X = df_sub[tf_list].values    # Shape: (num_samples, 12)

# Targets: 4 genes
y = df_sub[target_genes].values  # Shape: (num_samples, 4)

print("X shape:", X.shape)
print("y shape:", y.shape)


In [ ]:
# 4. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes: ", X_test.shape,  y_test.shape)


In [ ]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Verify the scaling
print("Mean of X_train_scaled:", X_train_scaled.mean(axis=0))
print("Std of X_train_scaled:", X_train_scaled.std(axis=0))

In [ ]:
# Convert to PyTorch tensors
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_t  = torch.tensor(X_test_scaled,  dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
y_test_t  = torch.tensor(y_test, dtype=torch.float32)


In [ ]:
# Define a custom Dataset
class RNARegressDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create Dataset instances
train_dataset = RNARegressDataset(X_train_t, y_train_t)
test_dataset  = RNARegressDataset(X_test_t,  y_test_t)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)


In [ ]:
# Define the neural network architecture
class TFRegressor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TFRegressor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)  # 4 outputs for ALB, AFP, APOA1, TTR

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Final output without activation for regression
        return x

# Instantiate the model
input_dim  = len(tf_list)      # 12
output_dim = len(target_genes) # 4
model = TFRegressor(input_dim, output_dim)


In [ ]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
# Define the training loop
def train_model(model, train_loader, criterion, optimizer, epochs=20):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=20)


## Loss is very high: Scale Targets and Retrain the Model

In [ ]:
# 6. Feature Scaling
from sklearn.preprocessing import StandardScaler

# Initialize scalers
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Fit scaler on training data and transform both training and testing data
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled  = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled  = scaler_y.transform(y_test)

# Verify the scaling
print("Mean of X_train_scaled:", X_train_scaled.mean(axis=0))
print("Std of X_train_scaled:", X_train_scaled.std(axis=0))
print("Mean of y_train_scaled:", y_train_scaled.mean(axis=0))
print("Std of y_train_scaled:", y_train_scaled.std(axis=0))


In [ ]:
# 7. Convert to PyTorch Tensors
import torch

X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_t  = torch.tensor(X_test_scaled,  dtype=torch.float32)
y_train_t = torch.tensor(y_train_scaled, dtype=torch.float32)
y_test_t  = torch.tensor(y_test_scaled, dtype=torch.float32)


In [ ]:
# 8. Create PyTorch Datasets and DataLoaders
from torch.utils.data import Dataset, DataLoader

class RNARegressDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = RNARegressDataset(X_train_t, y_train_t)
test_dataset  = RNARegressDataset(X_test_t,  y_test_t)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)


In [ ]:
# 9. Define the Multi-Output Regression Network
import torch.nn as nn
import torch.nn.functional as F

class TFRegressor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TFRegressor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)  # 4 outputs for ALB, AFP, APOA1, TTR

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Final output without activation for regression
        return x

input_dim  = len(tf_list)      # 12
output_dim = len(target_genes) # 4
model = TFRegressor(input_dim, output_dim)


In [ ]:
# 10. Initialize Weights (Optional)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

model.apply(init_weights)


In [ ]:
# 11. Set Loss and Optimizer
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
# 12. Define the Training Loop with Debugging
def train_model(model, train_loader, criterion, optimizer, epochs=20):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)

            # Debug: Check for NaN in loss
            if torch.isnan(loss):
                print(f"NaN loss detected at epoch {epoch+1}")
                return

            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}")


In [ ]:
# 13. Train the Model
train_model(model, train_loader, criterion, optimizer, epochs=20)


In [ ]:
# 14. Evaluate on Test Set
def evaluate_model(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            total_loss += loss.item()
    return total_loss / len(data_loader)

test_mse = evaluate_model(model, test_loader, criterion)
print(f"Test MSE: {test_mse:.4f}")


In [ ]:
# 15. (Optional) Compute R² for Each Gene
def compute_r2(model, data_loader, target_genes):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            y_pred = model(X_batch)
            all_preds.append(y_pred.numpy())
            all_targets.append(y_batch.numpy())

    all_preds   = np.concatenate(all_preds, axis=0)   # shape: (num_test_samples, 4)
    all_targets = np.concatenate(all_targets, axis=0) # same shape

    # Inverse transform to original scale if needed
    all_preds_orig = scaler_y.inverse_transform(all_preds)
    all_targets_orig = scaler_y.inverse_transform(all_targets)

    r2_vals = r2_score(all_targets_orig, all_preds_orig, multioutput='raw_values')
    print("R² per gene:", dict(zip(target_genes, r2_vals)))

    r2_mean = r2_score(all_targets_orig, all_preds_orig, multioutput='uniform_average')
    print(f"Mean R² across all 4 genes: {r2_mean:.3f}")

compute_r2(model, test_loader, target_genes)


## Alternative Approaches to Model Interpretation
Given the limitations with `DeepExplainer` for multi-output models, here are alternative methods to interpret your model:

a. Using SHAP's KernelExplainer
KernelExplainer is a model-agnostic explainer that works by approximating SHAP values through sampling. While it's computationally more intensive than DeepExplainer, it supports multi-output models.

Implementation Steps:
1. Prepare a Background Dataset: KernelExplainer requires a background dataset to simulate feature distributions.

2. Define a Prediction Function: This function should return predictions for the entire set of target genes.

3. Compute SHAP Values for Each Gene Separately: Since KernelExplainer doesn't inherently support multi-output explanations, you'll need to compute SHAP values for each gene individually.

In [ ]:
!pip install shap


In [ ]:
# Install SHAP if not already installed
!pip install shap

import shap
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure the model is on CPU
model.cpu()

# Define a prediction function for KernelExplainer
def predict_function(X):
    model.eval()
    with torch.no_grad():
        inputs = torch.tensor(X, dtype=torch.float32)
        outputs = model(inputs).numpy()
    return outputs

# Select a background dataset (e.g., a subset of training data)
background = X_train_scaled[:100]

# Initialize KernelExplainer for each target gene
explainer_dict = {}
shap_values_dict = {}

for i, gene in enumerate(target_genes):
    print(f"Computing SHAP values for {gene}...")
    # Define a function that returns only the current gene's prediction
    def single_output_predict(X):
        return predict_function(X)[:, i]

    explainer = shap.KernelExplainer(single_output_predict, background)
    shap_values = explainer.shap_values(X_test_scaled[:50], nsamples=100)  # Adjust nsamples as needed
    explainer_dict[gene] = explainer
    shap_values_dict[gene] = shap_values

    # Plot SHAP summary plot for the gene
    shap.summary_plot(shap_values, X_test_scaled[:50], feature_names=tf_list, show=False)
    plt.title(f'SHAP Summary Plot for {gene}')
    plt.show()


### Notes:

* Performance: KernelExplainer can be slow, especially with larger datasets. Adjust nsamples based on your computational resources.

* Sample Size: Using a subset of test data (e.g., first 50 samples) can speed up computation. Ensure this subset is representative.

* Visualization: The shap.summary_plot provides a global view of feature importance for each gene.

In [ ]:
# Assuming shap_values_dict and feature_importances are already computed
combined_importances = {}

for gene in target_genes:
    # SHAP importances (absolute mean SHAP values)
    shap_importances = np.abs(shap_values_dict[gene]).mean(axis=0)

    # Permutation importances
    perm_importances = feature_importances[gene]

    combined_importances[gene] = {
        'SHAP Importance': shap_importances,
        'Permutation Importance': perm_importances
    }

# Create a combined DataFrame
multi_importance_df = {}

for gene in target_genes:
    for i, tf in enumerate(tf_list):
        multi_importance_df[(tf, gene)] = [
            combined_importances[gene]['SHAP Importance'][i],
            combined_importances[gene]['Permutation Importance'][i]
        ]

# Convert to DataFrame
multi_importance_df = pd.DataFrame.from_dict(multi_importance_df, orient='index',
                                             columns=['SHAP Importance', 'Permutation Importance'])
multi_importance_df.index = pd.MultiIndex.from_tuples(multi_importance_df.index,
                                                      names=['Transcription Factor', 'Target Gene'])

# Pivot for heatmap
shap_heatmap = multi_importance_df.reset_index().pivot('Transcription Factor', 'Target Gene', 'SHAP Importance')
perm_heatmap = multi_importance_df.reset_index().pivot('Transcription Factor', 'Target Gene', 'Permutation Importance')

# Plot SHAP Heatmap
plt.figure(figsize=(12,10))
sns.heatmap(shap_heatmap, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('SHAP Importance of TFs for Each Target Gene', fontsize=16)
plt.xlabel('Target Gene', fontsize=14)
plt.ylabel('Transcription Factor', fontsize=14)
plt.tight_layout()
plt.show()

# Plot Permutation Importance Heatmap
plt.figure(figsize=(12,10))
sns.heatmap(perm_heatmap, annot=True, cmap='magma', fmt=".2f")
plt.title('Permutation Feature Importance of TFs for Each Target Gene', fontsize=16)
plt.xlabel('Target Gene', fontsize=14)
plt.ylabel('Transcription Factor', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
shap_threshold = 0.05  # Adjust based on your data
perm_threshold = 0.02  # Adjust based on your data


In [ ]:
import networkx as nx

# Initialize a graph
G = nx.Graph()

# Add gene and TF nodes
for gene in target_genes:
    G.add_node(gene, type='gene', color='red')
for tf in tf_list:
    G.add_node(tf, type='tf', color='blue')

# Add edges based on combined importance
for gene in target_genes:
    for tf in tf_list:
        shap_imp = combined_importances[gene]['SHAP Importance'][tf_list.index(tf)]
        perm_imp = combined_importances[gene]['Permutation Importance'][tf_list.index(tf)]

        # Define a combined importance metric
        combined_imp = shap_imp + perm_imp

        if combined_imp > (shap_threshold + perm_threshold):
            G.add_edge(tf, gene, weight=combined_imp)
